In [1]:
%matplotlib inline


# Step-by-step PyDESeq2 workflow

This notebook details all the steps of the PyDESeq2 pipeline.

It allows you to run the PyDESeq2 pipeline on the synthetic data provided
in this repository.

If this is your first contact with PyDESeq2, we recommend you first have a look at the
:doc:`standard workflow example <plot_minimal_pydeseq2_pipeline>`.
    :depth: 3

We start by importing required packages and setting up an optional path to save
results.


In [2]:
import os
import pickle as pkl

import numpy as np
import pandas as pd

from pydeseq2.dds import DeseqDataSet
from pydeseq2.default_inference import DefaultInference
from pydeseq2.ds import DeseqStats
from pydeseq2.utils import load_example_data

SAVE = False  # whether to save the outputs of this notebook

if SAVE:
    # Replace this with the path to directory where you would like results to be
    # saved
    OUTPUT_PATH = "../output_files/synthetic_example"
    os.makedirs(OUTPUT_PATH, exist_ok=True)  # Create path if it doesn't exist

## Data loading

Note that we are also explaining this step in the 'getting started' example.
To perform differential expression analysis (DEA), PyDESeq2 requires two types of
inputs:

  * A count matrix of shape 'number of samples' x 'number of genes', containing
    read counts (non-negative integers),
  * Metadata (or "column" data) of shape 'number of samples' x
    'number of variables', containing sample annotations that will be used
    to split the data in cohorts.

Both should be provided as [pandas dataframes](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html).

.. currentmodule:: pydeseq2

To illustrate the required data format, we load a synthetic example dataset
that may be
obtained through PyDESeq2's API using :func:`utils.load_example_data`.
You may replace it with your own dataset.



In [3]:
# counts_df = load_example_data(
#     modality="raw_counts",
#     dataset="synthetic",
#     debug=False,
# )

# metadata = load_example_data(
#     modality="metadata",
#     dataset="synthetic",
#     debug=False,
# )

In [4]:
cancer = pd.read_csv('./fpkm_BRCA_cancer.txt',sep='\t')
control = pd.read_csv('./fpkm_BRCA_control.txt',sep='\t')

In [5]:
counts_df = pd.merge(cancer,control,how='left',on='Row')
counts_df.index = counts_df['Row']
counts_df = counts_df.drop('Row',axis=1)
counts_df = counts_df.T
counts_df = counts_df*1e6
counts_df = counts_df.round(0)
counts_df

Row,1/2-SBSRNA4,A1BG,A1BG-AS1,A1CF,A2LD1,A2M,A2ML1,A2MP1,A4GALT,A4GNT,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,tAKR
BRCA_C_TCGA_A8_A0AB_01A_11R_A034_07,1569077.0,2148906.0,942109.0,8595.0,3954689.0,6.384240e+07,55999.0,56832.0,1905393.0,15416.0,...,19580397.0,2560212.0,7449721.0,6854794.0,1240125.0,8825680.0,50256612.0,7917441.0,17336064.0,9772.0
BRCA_C_TCGA_AO_A12H_01A_11R_A115_07,736707.0,5491541.0,875310.0,9865.0,1878134.0,1.231666e+08,29214.0,169588.0,2374061.0,35386.0,...,20957454.0,2953426.0,8577864.0,5617377.0,891397.0,6427020.0,38652356.0,6576281.0,97388811.0,22430.0
BRCA_C_TCGA_E9_A1NC_01A_21R_A26B_07,911287.0,5106914.0,409438.0,4464.0,1949028.0,3.001178e+08,32098530.0,442772.0,48621829.0,24021.0,...,18570275.0,1038197.0,1918970.0,1986346.0,841667.0,3187221.0,139789940.0,2674948.0,6534257.0,15226.0
BRCA_C_TCGA_A1_A0SM_01A_11R_A084_07,325704.0,8431866.0,824810.0,19329.0,1752101.0,9.307466e+07,21135.0,47188.0,7409132.0,32000.0,...,13938474.0,1570906.0,4161033.0,3552596.0,583841.0,8373484.0,95573306.0,4988366.0,16101755.0,10142.0
BRCA_C_TCGA_A2_A0D0_01A_11R_A00Z_07,412476.0,558804.0,74129.0,6214.0,550704.0,1.284723e+09,77284.0,755954.0,1660472.0,22289.0,...,25420277.0,758531.0,2231866.0,6343872.0,4232981.0,6380214.0,194522021.0,2414509.0,11290660.0,14128.0
BRCA_C_TCGA_A8_A06Y_01A_21R_A00Z_07,782255.0,7767458.0,1767657.0,9243.0,1547265.0,7.454857e+07,57478.0,0.0,4483627.0,0.0,...,12275942.0,2487835.0,6056757.0,4251710.0,1969316.0,4624306.0,46640777.0,7085624.0,44922601.0,0.0
BRCA_C_TCGA_C8_A1HO_01A_11R_A13Q_07,734457.0,5047419.0,862470.0,0.0,1386690.0,7.595501e+07,95316.0,39903.0,9743496.0,9020.0,...,28712579.0,2557017.0,6795227.0,5483331.0,1342736.0,8075512.0,33547722.0,4345835.0,16392007.0,11435.0
BRCA_C_TCGA_D8_A4Z1_01A_21R_A266_07,1581304.0,6344450.0,2049442.0,0.0,2335785.0,2.625879e+08,119382.0,339242.0,12923436.0,115028.0,...,9134687.0,1649714.0,4049119.0,5941577.0,1144732.0,6280072.0,139420123.0,11196446.0,14433229.0,20832.0
BRCA_C_TCGA_C8_A1HL_01A_11R_A137_07,722879.0,6614207.0,674556.0,8377.0,760738.0,2.954844e+08,9923.0,155081.0,3942793.0,0.0,...,70392699.0,1912222.0,4635797.0,3332957.0,4853017.0,5712343.0,79568851.0,2876738.0,11411124.0,19046.0
BRCA_C_TCGA_B6_A0RH_01A_21R_A115_07,215720.0,5844958.0,955374.0,27855.0,1117506.0,9.226308e+07,87987.0,12278.0,10516741.0,0.0,...,19438263.0,807573.0,2522135.0,2536195.0,1729745.0,3111473.0,126375977.0,2205105.0,7565721.0,10555.0


In [6]:
metadata = pd.DataFrame({
    'condition': ['A', 'A', 'A','A', 'A', 'A','A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B'],
    'group': ['A', 'A', 'A','A', 'A', 'A','A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B']
})
metadata.index = counts_df.index
metadata

,condition,group
BRCA_C_TCGA_A8_A0AB_01A_11R_A034_07,A,A
BRCA_C_TCGA_AO_A12H_01A_11R_A115_07,A,A
BRCA_C_TCGA_E9_A1NC_01A_21R_A26B_07,A,A
BRCA_C_TCGA_A1_A0SM_01A_11R_A084_07,A,A
BRCA_C_TCGA_A2_A0D0_01A_11R_A00Z_07,A,A
BRCA_C_TCGA_A8_A06Y_01A_21R_A00Z_07,A,A
BRCA_C_TCGA_C8_A1HO_01A_11R_A13Q_07,A,A
BRCA_C_TCGA_D8_A4Z1_01A_21R_A266_07,A,A
BRCA_C_TCGA_C8_A1HL_01A_11R_A137_07,A,A
BRCA_C_TCGA_B6_A0RH_01A_21R_A115_07,A,A


## 1. Read counts modeling
Read counts modeling with the :class:`DeseqDataSet
<dds.DeseqDataSet>` class

The :class:`DeseqDataSet <dds.DeseqDataSet>` class has two mandatory
arguments, ``counts`` and
``metadata``, as well as a set of optional keyword arguments, among which:

- ``design``: a string representing a
  [formulaic](https://github.com/matthewwardrop/formulaic) formula, or a design
  matrix (ndarray), that will be used to model the data.
- ``refit_cooks``: whether to refit cooks outliers – this is advised, in general.

<div class="alert alert-info"><h4>Note</h4><p>in the case of the provided synthetic data, there won't be any Cooks
  outliers.</p></div>



In [7]:
inference = DefaultInference(n_cpus=8)
dds = DeseqDataSet(
    counts=counts_df,
    metadata=metadata,
    design="~condition",  # compare samples based on the "condition"
    # column ("B" vs "A")
    refit_cooks=True,
    inference=inference,
)

### Compute normalization factors



In [8]:
dds.fit_size_factors()

dds.obsm["size_factors"]

Using None as control genes, passed at DeseqDataSet initialization


Fitting size factors...
... done in 0.08 seconds.



array([1.07029433, 0.98697556, 0.93381998, 0.86859398, 0.83829525,
       1.04177127, 1.02822039, 1.25835114, 0.9690605 , 0.86914191,
       0.8672654 , 1.05688881, 0.86792981, 1.08224724, 0.97612794,
       1.1227542 , 1.19794923, 1.1128547 , 1.1409546 , 1.05540343])

### Fit genewise dispersions



In [9]:
dds.fit_genewise_dispersions()

dds.varm["genewise_dispersions"]

Fitting dispersions...
... done in 73.99 seconds.



array([0.1913797 , 0.27971439, 0.33914722, ..., 0.15984186, 0.40342307,
       2.24996291])

### Fit dispersion trend coefficients



In [10]:
dds.fit_dispersion_trend()
dds.uns["trend_coeffs"]
dds.varm["fitted_dispersions"]

Fitting dispersion trend curve...
... done in 1.42 seconds.



array([ 0.84757107,  0.42668308,  0.92019066, ...,  0.37419474,
        0.32447134, 16.03112886])

### Dispersion priors



In [11]:
dds.fit_dispersion_prior()
print(
    f"logres_prior={dds.uns['_squared_logres']}, sigma_prior={dds.uns['prior_disp_var']}"
)

logres_prior=0.9386876093847428, sigma_prior=0.8211755946907114


### MAP Dispersions
The `fit_MAP_dispersions` method filters the genes for which dispersion
shrinkage is applied.
Indeed, for genes whose MLE dispersions are too high above the trend curve,
the original MLE value is kept.
The final values of the dispersions that are used for downstream analysis is
stored in `dds.dispersions`.



In [12]:
dds.fit_MAP_dispersions()
dds.varm["MAP_dispersions"]
dds.varm["dispersions"]

Fitting MAP dispersions...
... done in 82.09 seconds.



array([0.22903284, 0.29319199, 0.37940189, ..., 0.17673335, 0.3943119 ,
       2.74385644])

### Fit log fold changes
Note that in the `DeseqDataSet` object, the log-fold changes are stored in
natural
log scale, but that the results dataframe output by the `summary` method of
`DeseqStats` displays LFCs in log2 scale (see later on).



In [13]:
dds.fit_LFC()
dds.varm["LFC"]

Fitting LFCs...
... done in 42.61 seconds.



,Intercept,condition[T.B]
Row,,
1/2-SBSRNA4,13.563368,0.053487
A1BG,15.508771,-1.272528
A1BG-AS1,13.732469,-0.631841
A1CF,9.230507,-0.283161
A2LD1,14.353173,-0.020093
...,...,...
ZYG11B,15.642964,0.263837
ZYX,18.414258,0.147850
ZZEF1,15.444355,0.299513


### Calculate Cooks distances and refit
Note that this step is optional.



In [14]:
dds.calculate_cooks()
if dds.refit_cooks:
    # Replace outlier counts
    dds.refit()

Calculating cook's distance...
... done in 0.17 seconds.

Replacing 2199 outlier genes.

Fitting dispersions...
... done in 3.67 seconds.

Fitting MAP dispersions...
... done in 3.77 seconds.

Fitting LFCs...
... done in 4.38 seconds.



Save everything



In [15]:
if SAVE:
    with open(os.path.join(OUTPUT_PATH, "dds_detailed_pipe.pkl"), "wb") as f:
        pkl.dump(dds, f)

## 2. Statistical analysis
Statistical analysis with the :class:`DeseqStats <ds.DeseqStats>` class.
The `DeseqDataSet` class has two unique mandatory arguments, `dds`, which should
be a *fitted* `DeseqDataSet` object, and ``contrast``, which should be a list
of 3 strings of the form ``["factor", "tested_level", "reference_level"]`` or a
numerical contrast vector, as well as a set of optional keyword
arguments, among which:

- `alpha`: the p-value and adjusted p-value significance threshold
- `cooks_filter`: whether to filter p-values based on cooks outliers
- `independent_filter`: whether to perform independent filtering to correct
  p-value trends.



In [16]:
ds = DeseqStats(
    dds,
    contrast=np.array([0, 1]),
    alpha=0.05,
    cooks_filter=True,
    independent_filter=True,
)

### Wald tests



In [17]:
ds.run_wald_test()
ds.p_values

Running Wald tests...
... done in 13.20 seconds.



Row
1/2-SBSRNA4    8.026554e-01
A1BG           1.479918e-07
A1BG-AS1       2.180604e-02
A1CF           7.159804e-01
A2LD1          9.101791e-01
                   ...     
ZYG11B         6.608429e-02
ZYX            4.993414e-01
ZZEF1          1.111391e-01
ZZZ3           2.880425e-02
tAKR           1.297023e-01
Length: 23368, dtype: float64

### Cooks filtering
This is optional

<div class="alert alert-info"><h4>Note</h4><p>Note: in the case of the provided synthetic data, there won't be any
  outliers.</p></div>



In [18]:
if ds.cooks_filter:
    ds._cooks_filtering()
ds.p_values

Row
1/2-SBSRNA4    8.026554e-01
A1BG           1.479918e-07
A1BG-AS1       2.180604e-02
A1CF           7.159804e-01
A2LD1          9.101791e-01
                   ...     
ZYG11B         6.608429e-02
ZYX            4.993414e-01
ZZEF1          1.111391e-01
ZZZ3           2.880425e-02
tAKR           1.297023e-01
Length: 23368, dtype: float64

### P-value adjustment



In [19]:
if ds.independent_filter:
    ds._independent_filtering()
else:
    ds._p_value_adjustment()

ds.padj

Row
1/2-SBSRNA4    0.893578
A1BG           0.000003
A1BG-AS1       0.064099
A1CF           0.839646
A2LD1          0.954455
                 ...   
ZYG11B         0.154946
ZYX            0.676618
ZZEF1          0.231477
ZZZ3           0.080050
tAKR           0.259821
Name: 0, Length: 23368, dtype: float64

### Building a results dataframe
This dataframe is stored in the `results_df` attribute of the `DeseqStats`
class.



In [20]:
ds.summary()

Log2 fold change & Wald test p-value, contrast vector: [0 1]
                 baseMean  log2FoldChange     lfcSE      stat        pvalue  \
Row                                                                           
1/2-SBSRNA4  7.984829e+05        0.077166  0.308773  0.249912  8.026554e-01   
A1BG         3.480130e+06       -1.835870  0.349354 -5.255041  1.479918e-07   
A1BG-AS1     7.047811e+05       -0.911554  0.397411 -2.293730  2.180604e-02   
A1CF         8.945606e+03       -0.408515  1.122800 -0.363836  7.159804e-01   
A2LD1        1.694972e+06       -0.028988  0.256960 -0.112813  9.101791e-01   
...                   ...             ...       ...       ...           ...   
ZYG11B       7.156692e+06        0.380636  0.207109  1.837852  6.608429e-02   
ZYX          1.072757e+08        0.213303  0.315758  0.675526  4.993414e-01   
ZZEF1        5.988091e+06        0.432106  0.271237  1.593094  1.111391e-01   
ZZZ3         1.887140e+07       -0.885708  0.405144 -2.186155  2.88042

In [29]:
ds.results_df.to_csv('./resultDiff.csv')

Save everything if SAVE is set to True



In [21]:
if SAVE:
    with open(os.path.join(OUTPUT_PATH, "stat_results_detailed_pipe.pkl"), "wb") as f:
        pkl.dump(ds, f)

### LFC Shrinkage
For visualization or post-processing purposes, it might be suitable to perform
LFC shrinkage. This is implemented by the `lfc_shrink` method.



In [22]:
ds.lfc_shrink(coeff="condition[T.B]")

Fitting MAP LFCs...
/root/miniconda3/envs/streamlit/lib/python3.10/site-packages/pydeseq2/utils.py:1087: RuntimeWarning: overflow encountered in exp
  counts - (counts + size) / (1 + size * np.exp(-xbeta - offset))
/root/miniconda3/envs/streamlit/lib/python3.10/site-packages/pydeseq2/utils.py:1087: RuntimeWarning: overflow encountered in exp
  counts - (counts + size) / (1 + size * np.exp(-xbeta - offset))


Shrunk log2 fold change & Wald test p-value: condition[T.B]
                 baseMean  log2FoldChange     lfcSE      stat        pvalue  \
Row                                                                           
1/2-SBSRNA4  7.984829e+05   -7.182435e-06  0.001462  0.249912  8.026554e-01   
A1BG         3.480130e+06   -3.040818e-05  0.001155 -5.255041  1.479918e-07   
A1BG-AS1     7.047811e+05   -6.390870e-06  0.001263 -2.293730  2.180604e-02   
A1CF         8.945606e+03   -4.647959e-07  0.001351 -0.363836  7.159804e-01   
A2LD1        1.694972e+06   -1.739289e-04  0.001467 -0.112813  9.101791e-01   
...                   ...             ...       ...       ...           ...   
ZYG11B       7.156692e+06   -1.430199e-05  0.001548  1.837852  6.608429e-02   
ZYX          1.072757e+08    1.787403e+00  0.313083  0.675526  4.993414e-01   
ZZEF1        5.988091e+06    6.258759e-05  0.001568  1.593094  1.111391e-01   
ZZZ3         1.887140e+07   -3.503314e-04  0.001383 -2.186155  2.880425

... done in 87.87 seconds.



Save everything



In [23]:
if SAVE:
    with open(os.path.join(OUTPUT_PATH, "shrunk_results_detailed_pipe.pkl"), "wb") as f:
        pkl.dump(ds, f)